### Импорты

In [1]:
from LDANewsModel import LDANewsModel
from pyspark import SparkContext
from pyspark.sql import DataFrame, SparkSession

### Сессия Spark

In [2]:
sc = SparkContext('local', 'LDAModel')
spark = SparkSession(sparkContext=sc)

# LDA на экономических новостях

## Загружаем данные

In [3]:
news_data = spark.read.parquet('data/russian_news.parquet')
news_data.show(10)

+---+--------------------+
| id|                text|
+---+--------------------+
|  1|МОСКВА, 17 ноября...|
|  3|ДУБАЙ, 17 ноя - Р...|
|  4|МИНСК, 17 ноя - Р...|
|  5|МИНСК, 17 ноя — Р...|
|  6|МОСКВА, 17 ноя — ...|
|  7|ДУБАЙ, 17 ноя - Р...|
|  8|ДУБАЙ, 17 ноя - Р...|
|  9|ДУБАЙ, 17 ноя - Р...|
| 10|МОСКВА, 17 ноября...|
| 52|МОСКВА, 15 ноя - ...|
+---+--------------------+
only showing top 10 rows



## LDA модель


In [4]:
lda_model = LDANewsModel(news_data)
lda_results, vocab_model = lda_model.fit(num_topics=15)

Смотрим, что получилось

In [5]:
topics = lda_results.describeTopics(maxTermsPerTopic=15)
vocabulary = vocab_model.vocabulary
topics.show()
topics_rdd = topics.rdd
topics_words = topics_rdd.map(lambda row: row['termIndices']).map(
    lambda idx_list: [vocabulary[idx] for idx in idx_list]
).collect()

for idx, topic in enumerate(topics_words):
    print(f'Тема {idx + 1}')
    print('----------------')
    for word in topic:
        print(word)
    print('----------------')

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[26, 32, 73, 50, ...|[0.01588410760535...|
|    1|[5, 13, 42, 3868,...|[0.00299402985683...|
|    2|[169, 137, 198, 1...|[0.01297650767635...|
|    3|[1599, 65, 101, 9...|[0.00188165207989...|
|    4|[979, 636, 657, 1...|[0.03318234126764...|
|    5|[3, 6, 0, 14, 11,...|[0.00881608807601...|
|    6|[2, 187, 244, 66,...|[0.02003024103961...|
|    7|[273, 109, 71, 19...|[0.01879324834551...|
|    8|[26, 32, 399, 114...|[0.00315638687166...|
|    9|[24, 111, 34, 8, ...|[0.00771866534802...|
|   10|[491, 368, 254, 7...|[0.02369333970777...|
|   11|[16, 18, 83, 13, ...|[0.00865659733646...|
|   12|[25, 102, 44, 101...|[0.01537160764892...|
|   13|[19, 185, 71, 125...|[0.01152187038150...|
|   14|[681, 694, 1098, ...|[0.03244937782702...|
+-----+--------------------+--------------------+

Тема 1
----------------
индекс
пункт
торг
баррел


## Сохраняем модель

In [6]:
lda_results.save('weights/LDA_model_2')